In [1]:
import networkx as nx
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from IF import *
from proutils import *
import pandas as pd
model = LinearModel()
model.load_state_dict(torch.load('data/main_model.pth'))
model.eval()  # Set the model to evaluation mode
ifem=np.load("data/influence_scores.npy")
train_embeddings = torch.load('data/train_embeddings.pt')
train_labels = torch.load('data/train_labels.pt')
test_embeddings = torch.load('data/test_embeddings.pt')
test_labels = torch.load('data/test_labels.pt')

/tmp/ipykernel_710589/4040735843.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('data/main_model.pth'))
/tmp/ipykernel_710589/404073584

In [ ]:
def aide(inf, train_embeddings, test_embeddings):

    aide_emb = []
    for i, emb in enumerate(inf):
        # Sort indices based on embedding values
        ind = emb.argsort()
        
        # Top 10 positive and negative indices
        top_pos_indices = ind[-10:]
        top_neg_indices = ind[:10]

        # Compute cosine similarity for positive and negative indices
        pos_sim = cosine_similarity(train_embeddings[top_pos_indices], test_embeddings[i].reshape(1, -1)).flatten()
        neg_sim = cosine_similarity(train_embeddings[top_neg_indices], test_embeddings[i].reshape(1, -1)).flatten()

        # Select top 5 positive and negative indices based on cosine similarity
        pos_idx = top_pos_indices[pos_sim.argsort()[-5:]]
        neg_idx = top_neg_indices[neg_sim.argsort()[-5:]]

        # Combine positive and negative indices
        aide_emb.append(np.append(pos_idx,neg_idx))

    return aide_emb

In [2]:
aide_em=aide(ifem, train_embeddings, test_embeddings, 10)

In [ ]:
G = nx.Graph()
for i, idx in enumerate(aide_em):
    G.add_node(i, bipartite=0)
    for j in idx:
        G.add_node(f'ex-{j}', bipartite=1)
        G.add_edge(i, f'ex-{j}', weight=0.5)
# nx.write_gml(G, "data/bigraph_aide.pkl")


In [ ]:
aide_em=aide(ifem, train_embeddings, test_embeddings, 10)

In [ ]:
G[0]['ex-1227']['weight']

In [ ]:
degree_centrality = nx.degree_centrality(G)

In [ ]:
top_k_nodes = sorted(degree_centrality, key=degree_centrality.get, reverse=True)[:20]

In [ ]:
tops=list(map(lambda x: int(x.split('-')[1]), top_k_nodes))

In [ ]:
import matplotlib.pyplot as plt

degrees=[G.degree(i) for i in G.nodes if G.nodes[i]['bipartite']==1]
plt.hist(degrees, bins=130)
plt.xticks(fontsize=14)
plt.ticklabel_format(style='sci', axis='x')
plt.yticks(fontsize=13)
plt.xlabel("Centrality Degree", fontsize=14)
plt.ylabel("Frequency", fontsize=14)
# plt.savefig('inf_cnn_sortsum.eps', format='eps')
plt.show()

In [ ]:
nodes_0 = [n for n, d in G.nodes(data=True) if d["bipartite"] == 0]
test_embeddings[nodes_0]

In [ ]:
from sklearn.cluster import SpectralClustering

def find_representative_samples(bipartite_graph, bipartite=0):

    # Extract nodes and neighbors
    nodes_0 = [n for n, d in bipartite_graph.nodes(data=True) if d["bipartite"] == bipartite]
    neighbors = {node: set(bipartite_graph.neighbors(node)) for node in nodes_0}
    
    # Compute pairwise connection strengths
    num_nodes = len(nodes_0)
    connection_matrix = np.zeros((num_nodes, num_nodes))
    
    for i, node1 in enumerate(nodes_0):
        for j, node2 in enumerate(nodes_0):
            if i != j:
                # Compute shared neighbors count
                connection_matrix[i, j] = len(neighbors[node1] & neighbors[node2])
    
    # Compute semantic similarity
    similarity_matrix = cosine_similarity(test_embeddings[nodes_0])
    
    # Combine connection strength and semantic similarity (prioritizing connections)
    combined_matrix = connection_matrix + 0.5 * similarity_matrix
    
    # Perform clustering
    clustering = SpectralClustering(
        n_clusters=20,  
        affinity="precomputed",
        random_state=42
    )
    labels = clustering.fit_predict(combined_matrix)
    
    # Select representatives
    representatives = []
    for cluster_id in np.unique(labels):
        cluster_nodes = [nodes_0[i] for i in range(num_nodes) if labels[i] == cluster_id]
        # Select the node with the maximum connections
        representative = max(cluster_nodes, key=lambda n: sum(connection_matrix[nodes_0.index(n)]))
        representatives.append(representative)
    
    return representatives


In [ ]:
protos=find_representative_samples(G, bipartite=0)

In [ ]:
test_labels[[2866,
 7256,
 2715,
 299,
 9000,
 3438,
 6279,
 9554,
 8379,
 9908,
 291,
 7721,
 34,
 3717,
 7888,
 8565,
 4415,
 871,
 4160,
 9136]]

In [ ]:
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
import torchvision.transforms.functional as F

def display_cifar10_images(indices, train=False):
    """
    Display CIFAR-10 images based on a list of indices and train/test flag.
    
    Args:
        indices (list of int): List of indices to display.
        train (bool): If True, load images from the training set. If False, use the test set.
    """
    # Define transformations for the dataset
    transform = transforms.Compose([
        transforms.ToTensor()
    ])
    
    # Load CIFAR-10 dataset (train or test)
    cifar10 = datasets.CIFAR10(root='./data', train=train, download=True, transform=transform)
    
    # Classes in CIFAR-10
    classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    
    # Number of images to display
    num_images = len(indices)
    
    # Set up a grid for the images
    plt.figure(figsize=(12, num_images * 2))  # Adjust figure size based on the number of images
    for i, index in enumerate(indices):
        image, label = cifar10[index]  # Get image and label at the given index
        
        # Resize the image for better display quality
        resized_image = F.resize(image, size=(128, 128))  # Resize to 128x128
        
        # Convert tensor to numpy format for visualization
        resized_image = resized_image.permute(1, 2, 0)  # Rearrange channels for matplotlib
        
        # Plot each image in a grid
        plt.subplot(1, num_images, i + 1)
        plt.imshow(resized_image)
        plt.title(f"{classes[label]}")
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Example Usage
indices_to_display = [2] # List of indices to display
display_cifar10_images(indices_to_display, train=False)  # Display test images


In [ ]:
test_labels[2]